Домашнее задание по теме «Python для анализа данных: pandas»

**Задание 1**

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце 'region' пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим 'undefined'.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

    'Центр': ['москва', 'тула', 'ярославль'],

    'Северо-Запад': ['петербург', 'псков', 'мурманск'],

    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']

}

Результат классификации запишите в отдельный столбец region.


In [1]:
import pandas as pd

In [2]:
kw = pd.read_csv('/home/lena/Netology/Python/Python_6_2019-03-18/keywords.csv')
kw.head()

,keyword,shows
0,вк,64292779
1,одноклассники,63810309
2,порно,41747114
3,ютуб,39995567
4,вконтакте,21014195


In [3]:
geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [4]:
for i in geo_data.keys():
    for j in geo_data[i]:
        print(i, j)

Центр москва
Центр тула
Центр ярославль
Северо-Запад петербург
Северо-Запад псков
Северо-Запад мурманск
Дальний Восток владивосток
Дальний Восток сахалин
Дальний Восток хабаровск


In [5]:
def geo_classif(row):
    for i in geo_data.keys():
        for j in geo_data[i]:
            if j in row['keyword']:
                return i 
    return 'undefined'

In [6]:
kw['region'] = kw.apply(geo_classif, axis=1)

In [7]:
kw.region.value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

**Задание 2**

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
    - оценка 2 и меньше - низкий рейтинг
    - оценка 4 и меньше - средний рейтинг
    - оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [8]:
ratings = pd.read_csv('/home/lena/Netology/Python/Python_6_2019-03-18/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [9]:
ratings_filtered = ratings.groupby('movieId').mean()[['rating']]
ratings_filtered = ratings_filtered.reset_index()
ratings_filtered.head()

,movieId,rating
0,1,3.872470
1,2,3.401869
2,3,3.161017
3,4,2.384615
4,5,3.267857


In [10]:
def classif(row):
    if row['rating'] <= 2:
        return 'низкий рейтинг'
    elif row['rating'] > 2 and row['rating'] <=4:
        return 'средний рейтинг'
    elif row['rating'] >= 4.5 and row['rating'] <= 5:
        return 'высокий рейтинг'
    else:
        return 'undefined'

In [11]:
ratings_filtered['class'] = ratings_filtered.apply(classif, axis=1)

In [12]:
ratings_filtered.groupby('class').count()[['movieId']]

,movieId
class,
undefined,604
высокий рейтинг,668
низкий рейтинг,973
средний рейтинг,6821


In [13]:
joined = ratings.merge(ratings_filtered, on='movieId', how='left')
joined.head()

,userId,movieId,rating_x,timestamp,rating_y,class
0,1,31,2.5,1260759144,3.178571,средний рейтинг
1,1,1029,3.0,1260759179,3.702381,средний рейтинг
2,1,1061,3.0,1260759182,3.545455,средний рейтинг
3,1,1129,2.0,1260759185,3.312500,средний рейтинг
4,1,1172,4.0,1260759205,4.260870,undefined


**Задание 3**

Посчитайте среднее значение Lifetime киноманов (пользователи, которые поставили 100 и более рейтингов). Под Lifetime понимается разница между максимальным и минимальным значением timestamp для каждого пользователя. Ответ дайте в днях.


In [14]:
ratings = pd.read_csv('/home/lena/Netology/Python/Python_6_2019-03-18/ml-latest-small/ratings.csv')

In [15]:
ratings_filtered = ratings.groupby('userId').count()

In [16]:
ratings_filtered = ratings_filtered[ratings_filtered['movieId'] > 100]
ratings_filtered = ratings_filtered.reset_index()

In [17]:
list_ = ratings_filtered['userId'].tolist()

In [18]:
user_100 = ratings[ratings['userId'].isin(list_)]

In [19]:
user_100m = user_100.groupby('userId').agg(['min', 'max'])['timestamp']
user_100m.head()

,min,max
userId,,
4,949778714,949982274
8,1154389340,1154474527
15,997937239,1469330735
17,1127468587,1127476640
19,855190091,855195373


In [20]:
user_100m['Lifetime'] = (user_100m['max']-user_100m['min'])/(60*60*24)

In [21]:
user_100m.head()

,min,max,Lifetime
userId,,,
4,949778714,949982274,2.356019
8,1154389340,1154474527,0.985961
15,997937239,1469330735,5455.943241
17,1127468587,1127476640,0.093206
19,855190091,855195373,0.061134


**Задание 4**

Есть мнение, что "раньше снимали настоящее кино, не то что сейчас". Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из материалов занятия. Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:
    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год

3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец 'year'

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца 'year' и отсортируйте результат по убыванию рейтинга
 

In [22]:
years = list(range(1950,2011))
print(years)

[1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958, 1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969, 1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010]


In [23]:
ratings = pd.read_csv('/home/lena/Netology/Python/Python_6_2019-03-18/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [24]:
movies = pd.read_csv('/home/lena/Netology/Python/Python_6_2019-03-18/ml-latest-small/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [25]:
def production_year(row):
    for i in years:
        if str(i) in row['title']:
            return str(i)
    return '1900'

In [26]:
movies['year'] = movies.apply(production_year, axis=1)
movies.head()

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [27]:
joined = ratings.merge(movies, on='movieId', how='left')
joined.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,31,2.5,1260759144,Dangerous Minds (1995),Drama,1995
1,1,1029,3.0,1260759179,Dumbo (1941),Animation|Children|Drama|Musical,1900
2,1,1061,3.0,1260759182,Sleepers (1996),Thriller,1996
3,1,1129,2.0,1260759185,Escape from New York (1981),Action|Adventure|Sci-Fi|Thriller,1981
4,1,1172,4.0,1260759205,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama,1989


In [28]:
year_rating = joined.groupby('year').mean()[['rating']].sort_values('rating', ascending=False)
year_rating = year_rating.reset_index()
year_rating.head()

,year,rating
0,1957,4.014241
1,1972,4.011136
2,1952,4.000000
3,1954,3.994220
4,1951,3.983539
